In [2]:
import pandas as pd
pd.options.display.max_columns=300
tsv = pd.read_csv("/Users/hollywhittome/Downloads/Supplemental_file1_neuron_annotations(1).tsv", sep="\t", low_memory=False)

from fafbseg import flywire

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/usr/local/anaconda3/lib/python3.11/site-packages/python_jsonschema_objects/__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


In [10]:
orns = tsv.loc[tsv.hemibrain_type.str.startswith("ORN", na=False)]
orns.head()

known_nt
acetylcholine                                                                       31744
glutamate                                                                            6884
gaba, nitric oxide                                                                   3136
acetylcholine, allatostatin-c                                                        3005
acetylcholine, sNPF, sparkly                                                         2490
acetylcholine, Nplp1, sNPF, sparkly                                                  1771
glutamate, gaba                                                                      1569
acetylcholine, allatostatin-a                                                        1506
gaba, nitric oxide, allatostatin-c                                                   1456
gaba                                                                                 1399
histamine                                                                            1324
a

In [8]:
da1 = 720575940654849697#ORN_DA1
dn = 720575940650929273
dm1 = 720575940625821758#ORN_DM1
edges = flywire.get_connectivity([dn, da1])
downstream_conns = edges[edges.pre.isin([dn,da1])].drop(columns = "pre")
downstream_conns

Using materialization version 783.


,post,weight
40,720575940627096232,16
41,720575940620402902,16
48,720575940622838154,15
68,720575940629938487,11
96,720575940631535073,9
...,...,...
4727,720575940473667228,1
4728,720575940473676188,1
4729,720575940473676700,1
4730,720575940473691804,1


In [27]:
downstream_nts = downstream_conns.merge(tsv[['root_id', 'hemibrain_type', 'top_nt', 'top_nt_conf', 'known_nt']], how = 'left',
                left_on = 'post', right_on = 'root_id').drop(columns = ['root_id'])
downstream_nts.head()

,post,weight,hemibrain_type,top_nt,top_nt_conf,known_nt
0,720575940620402902,16,NaN,gaba,0.849847,NaN
1,720575940627096232,16,NaN,gaba,0.806208,NaN
2,720575940622838154,15,Giant Fiber,acetylcholine,0.317149,NaN
3,720575940629938487,11,SAD021_c,gaba,0.722425,NaN
4,720575940621146771,9,NaN,gaba,0.828461,NaN


In [28]:
dns = downstream_nts.loc[downstream_nts.hemibrain_type.str.startswith("DN", na=False)]
dns

,post,weight,hemibrain_type,top_nt,top_nt_conf,known_nt
301,720575940622673860,2,DNp06,acetylcholine,0.503946,NaN
313,720575940618594022,2,DNp42,acetylcholine,0.898806,NaN
455,720575940612203482,1,DNp05,acetylcholine,0.844788,NaN
1225,720575940632903737,1,DNp34,acetylcholine,0.909321,NaN
2077,720575940625539984,1,DNp49,glutamate,0.703236,NaN


In [19]:
#orn = da1
for orn in list(orns.root_id):
    edges = flywire.get_connectivity(orn)
    downstream_conns = edges[edges.pre == orn].drop(columns = "pre")
    downstream_conns.head()
    downstream_nts = downstream_conns.merge(tsv[['root_id', 'hemibrain_type', 'top_nt', 'top_nt_conf', 'known_nt']], how = 'left',
                    left_on = 'post', right_on = 'root_id').drop(columns = ['root_id'])
    downstream_nts.head()
    dns = downstream_nts.loc[downstream_nts.hemibrain_type.str.startswith("DN", na=False)]
    if dns.shape[0] == 0:
        print(orn, dns.shape[0])
    else:
        print(orn, dns)

Using materialization version 783.
720575940619419757 0
Using materialization version 783.
720575940629918762 0
Using materialization version 783.
720575940629728732 0
Using materialization version 783.
720575940620028788 0
Using materialization version 783.
720575940632224031 0
Using materialization version 783.
720575940628800277 0
Using materialization version 783.
720575940631217782 0
Using materialization version 783.
720575940635146926 0
Using materialization version 783.
720575940620071872 0
Using materialization version 783.
720575940626779644 0
Using materialization version 783.
720575940635757936 0
Using materialization version 783.
720575940626976522 0
Using materialization version 783.
720575940628429056 0
Using materialization version 783.
720575940622651389 0
Using materialization version 783.
720575940643977070 0
Using materialization version 783.
720575940607927817 0
Using materialization version 783.
720575940642284704 0
Using materialization version 783.
7205759406198

In [11]:
def find_downstream_nts(type):
    type_neurons = tsv.loc[tsv.hemibrain_type.str.startswith(type, na=False)]
    type_ids = list(type_neurons.root_id)
    edges = flywire.get_connectivity(type_ids)
    downstream_conns = edges[edges.pre.isin(type_ids)].drop(columns = "pre")
    downstream_nts = downstream_conns.merge(tsv[['root_id', 'hemibrain_type', 'top_nt', 'top_nt_conf', 'known_nt']], how = 'left',
                        left_on = 'post', right_on = 'root_id').drop(columns = ['root_id'])
    conf_downstream_nts = downstream_nts[downstream_nts.top_nt_conf > 0.5]
    synapses_per_nt = conf_downstream_nts.groupby(by=["top_nt"])["weight"].sum()
    return synapses_per_nt

In [14]:
type = "ORN_DA1"
result = find_downstream_nts(type)

Using materialization version 783.


Fetching connectivity:   0%|          | 0/5 [00:00<?, ?it/s]

In [15]:
result

top_nt
acetylcholine    6793
dopamine            1
gaba             1816
glutamate         112
octopamine         15
serotonin         374
Name: weight, dtype: int64